# Price–News Join Analysis

Analyze the GDELT–OHLCV join table: **news on day t → prices on day t+1** (next trading day).  
We evaluate and validate the data, then compute mean and median sentiment (and optional price metrics) per ticker per day.



In [ ]:
import pandas as pd
from pathlib import Path

# Find project root
current = Path.cwd()
while not (current / "data").exists() and current != current.parent:
    current = current.parent
PROJECT_ROOT = current
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
INPUT_PATH = PROCESSED_DIR / "gdelt_ohlcv_join.csv"

df = pd.read_csv(
    INPUT_PATH,
    parse_dates=["seendate", "article_date", "price_date"],
)
# Required for analysis
required = ["sentiment_score", "ticker", "article_date", "price_date"]
missing = [c for c in required if c not in df.columns]
assert not missing, f"Missing columns: {missing}"
print(f"Loaded {len(df):,} rows from {INPUT_PATH.name}")
print(f"Columns: {list(df.columns)}")

## 1A. Alternative Join (Trading-Day Indexed)

Alternative approach for comparison:
- Use `prices_daily_accumulated.csv` as the left/base table (one row per trading day per ticker).
- Left-join `gdelt_articles_with_sentiment.csv` on `ticker` and calendar date.
- This keeps the trading-day index intact and attaches any same-day news rows.

This is separate from the existing `gdelt_ohlcv_join.csv` logic (`news t -> prices t+1`) and is intended for validation/EDA comparisons.

In [ ]:
# Trading-day indexed left join: prices (left) <- gdelt accumulated
prices_path = PROCESSED_DIR / "prices_daily_accumulated.csv"
gdelt_acc_path = PROCESSED_DIR / "gdelt_articles_with_sentiment.csv"

prices_left = pd.read_csv(prices_path, parse_dates=["date"])
gdelt_acc = pd.read_csv(gdelt_acc_path, parse_dates=["seendate"])

# Normalize join keys to calendar day + ticker
prices_left["trading_date"] = prices_left["date"].dt.tz_localize(None).dt.normalize()
gdelt_acc["article_date"] = pd.to_datetime(gdelt_acc["seendate"], utc=True).dt.tz_localize(None).dt.normalize()

prices_left["ticker"] = prices_left["ticker"].astype(str).str.upper().str.strip()
gdelt_acc["ticker"] = gdelt_acc["ticker"].astype(str).str.upper().str.strip()

alt_join = prices_left.merge(
    gdelt_acc,
    how="left",
    left_on=["trading_date", "ticker"],
    right_on=["article_date", "ticker"],
    suffixes=("_price", "_news"),
)

print("Alternative join built:")
print(f"  prices rows (left base): {len(prices_left):,}")
print(f"  joined rows: {len(alt_join):,}")
print(f"  trading date range: {prices_left['trading_date'].min().date()} -> {prices_left['trading_date'].max().date()}")

# Coverage diagnostics: how many trading-day x ticker rows have at least one matched article
coverage = (
    alt_join.groupby(["trading_date", "ticker"], as_index=False)
    .agg(has_news=("url", lambda s: s.notna().any()), article_rows=("url", "count"))
)

print(f"  unique (trading_date, ticker) pairs: {len(coverage):,}")
print(f"  pairs with >=1 matched article: {int(coverage['has_news'].sum()):,}")
print(f"  pairs with 0 matched articles: {int((~coverage['has_news']).sum()):,}")

print("\nSample rows from alternative join:")
display(
    alt_join[["trading_date", "ticker", "close", "seendate", "title", "sentiment_score"]]
    .sort_values(["trading_date", "ticker", "seendate"], na_position="last")
    .head(20)
)

# Keep as named object for downstream analysis cells if needed
alt_join_df = alt_join

In [ ]:
merged = alt_join_df.sort_values(['ticker', 'trading_date'])

# Create next day's close based on each ticker's last close
merged['next_close'] = merged.groupby('ticker')['close'].shift(-1)
# Calculate next day's return
merged['next_return'] = (merged['next_close'] - merged['close']) / merged['close']
# Calculate absolute return
merged['abs_return'] = merged['next_return'].abs()

daily = merged.groupby(['ticker', 'trading_date']).agg(
    mean_sentiment=('sentiment_score', 'mean'),
    median_sentiment=('sentiment_score', 'median'),
    sentiment_count=('sentiment_present', 'sum'),
    article_count=('title', 'count'),
    next_return=('next_return', 'first'),
    abs_return=('abs_return', 'first')
).reset_index()
daily

# Questions:
## Does sentiment predict price direction?
## sentiment predict volatility?
## sentiment bucket analysis
## news volume effect
## disagreement signal (sentiment range vs abs return)

## 1. Data evaluation and validation

**First objective: validate the matching between prices and news** (join integrity, then schema and coverage).

In [ ]:
# Schema and shape
print("Shape:", df.shape)
print("\nDtypes:")
print(df.dtypes)
print("\nKey columns present:")
key_cols = ["article_date", "price_date", "ticker", "sentiment_score", "next_close", "next_volume"]
for c in key_cols:
    print(f"  {c}: {c in df.columns}")

### 1.1 Validate price–news matching

Check that **price_date** is the next trading day after **article_date** and that attached prices match the source OHLCV.

In [ ]:
# 1) price_date must be strictly after article_date (next trading day)
df["_gap_days"] = (df["price_date"] - df["article_date"]).dt.days
bad_order = (df["_gap_days"] <= 0).sum()
print("1) Article date → price date (next trading day)")
# Display number of misaligned rows (s/b 0)
print(f"   Rows where price_date ≤ article_date: {bad_order} (expect 0)")
# Unit test for correct matching
assert bad_order == 0, "Every row must have price_date > article_date"
print("   ✓ All rows have price_date after article_date")
print("\n   Calendar-day gap (article_date to price_date):")
print(df["_gap_days"].value_counts().sort_index().to_string())

In [ ]:
# 2) Within join: each (price_date, ticker) should have exactly one set of next_* values (no conflicts)
price_cols = [c for c in df.columns if c.startswith("next_")]
check = df.groupby(["price_date", "ticker"])[price_cols].nunique()
max_per_col = check.max()
conflicts = (check > 1).any(axis=1).sum()
print("2) One price per (price_date, ticker)")
print(f"   Unique (price_date, ticker) pairs: {len(check):,}")
print(f"   Pairs with conflicting next_* values: {conflicts} (expect 0)")
# Unit test for unique prices per (price_date, ticker)
assert conflicts == 0, "⚠ Some (price_date, ticker) have multiple different prices — investigate"
print("   ✓ All rows for same (price_date, ticker) have identical next_* values")

In [ ]:
# 3) Cross-check: join next_* values vs source OHLCV (prices_daily_accumulated)
ohlcv_path = PROCESSED_DIR / "prices_daily_accumulated.csv"
if ohlcv_path.exists():
    ohlcv = pd.read_csv(ohlcv_path, parse_dates=["date"])
    # One row per (date, ticker) in join; take first next_* per (price_date, ticker)
    join_prices = df.groupby(["price_date", "ticker"])["next_close"].first().reset_index()
    join_prices = join_prices.rename(columns={"price_date": "date", "next_close": "join_close"})
    merged = join_prices.merge(ohlcv[["date", "ticker", "close"]], on=["date", "ticker"], how="left")
    merged["match"] = merged["join_close"].round(6) == merged["close"].round(6)
    mismatches = (~merged["match"]).sum()
    missing = merged["close"].isna().sum()
    print("3) Join vs source OHLCV (next_close vs close)")
    print(f"   (price_date, ticker) pairs checked: {len(merged):,}")
    print(f"   Mismatches (join next_close ≠ OHLCV close): {mismatches}")
    print(f"   Missing in OHLCV: {missing}")
    # Unit test for matching prices
    assert mismatches == 0 and missing == 0, "   ⚠ Review mismatches or missing dates"
    print("   ✓ All join prices match source OHLCV")
else:
    print("3) Skip cross-check (prices_daily_accumulated.csv not found)")

### 1.2 Schema, date ranges, and coverage

Schema, date ranges, missing values, and ticker coverage (for downstream aggregation).

In [ ]:
# Date ranges and join alignment
art_min, art_max = df["article_date"].min(), df["article_date"].max()
price_min, price_max = df["price_date"].min(), df["price_date"].max()
print("Article date range:", art_min.date(), "to", art_max.date())
print("Price date range: ", price_min.date(), "to", price_max.date())
print("\nExpected: price_date = next trading day after article_date (weekends/holidays skipped).")
# Spot-check: article_date and price_date should differ by 1–3 calendar days (Fri→Mon = 3)
df["days_to_next"] = (df["price_date"] - df["article_date"]).dt.days
print("\nCalendar days from article_date to price_date (sample):")
print(df["days_to_next"].value_counts().head(10))

In [ ]:
# Missing values in columns used for aggregation
agg_cols = ["sentiment_score", "ticker", "article_date", "price_date"]
if "next_close" in df.columns:
    agg_cols.append("next_close")
missing = df[agg_cols].isna().sum()
print("Missing values (columns used for mean/median per ticker per day):")
print(missing[missing > 0] if missing.any() else "  None")
print("\nRows with any missing in these columns:", df[agg_cols].isna().any(axis=1).sum())

In [ ]:
# Ticker coverage: articles and (article_date, ticker) pairs
print("Articles per ticker:")
print(df["ticker"].value_counts().sort_index())
print("\nUnique (article_date, ticker) pairs per ticker = distinct calendar days with ≥1 article:")
# Per ticker, count distinct article_date (same as count of (article_date, ticker) per ticker)
unique_days_per_ticker = df.groupby("ticker")["article_date"].nunique()
print(unique_days_per_ticker.to_string())
print("\n(Multiple articles per day per ticker is expected; we aggregate to mean/median per (date, ticker) later.)")

In [ ]:
# Compute mean/median sentiment per ticker per day
sentiment_per_ticker_per_day = df.groupby(["ticker", "price_date"])["sentiment_score"].agg(["mean", "median"])
print("\nMean and median sentiment per ticker per day:")
print(sentiment_per_ticker_per_day.head())

# Sanity check aggregated outputs across tickers


## 2. Sentiment summary statistics per ticker per day

Aggregate to mean and median sentiment per (ticker, day).

In [ ]:
# Aggregate: one row per (article_date, ticker)
daily = df.groupby(["article_date", "ticker"]).agg(
    sentiment_mean=("sentiment_score", "mean"),
    sentiment_median=("sentiment_score", "median"),
    article_count=("sentiment_score", "count"),
).reset_index()
if "next_close" in df.columns:
    daily["next_close"] = df.groupby(["article_date", "ticker"])["next_close"].first().values
if "next_volume" in df.columns:
    daily["next_volume"] = df.groupby(["article_date", "ticker"])["next_volume"].first().values

print("Daily summary (first rows):")
daily.head(10)

## 3. Monday-only analysis: sentiment per ticker per week

Filter to **Mondays only** (article_date), then group by week (7-day increments starting from **2026-01-12** as week 1). Compute summary statistics per ticker per week (open-ended; weeks 1+).

In [ ]:
# Filter to Mondays only (dayofweek: Monday=0)
mondays = daily[daily["article_date"].dt.dayofweek == 0].copy()
print(f"Total rows: {len(daily):,}")
print(f"Mondays only: {len(mondays):,} ({100*len(mondays)/len(daily):.1f}%)")
print(f"\nMonday dates in data:")
monday_dates = sorted(mondays["article_date"].dt.date.unique())
print(monday_dates)
print(f"\nExpected Mondays (if all weeks present):")
week_start = pd.Timestamp("2026-01-12")
for w in range(1, 6):  # weeks 1-5 based on data up to 2/9
    expected_monday = week_start + pd.Timedelta(days=7*(w-1))
    print(f"  Week {w}: {expected_monday.date()}")
    if expected_monday.date() not in monday_dates:
        print(f"    ⚠ MISSING from data")

In [ ]:
# Assign week number (increments of 7 days starting from 2026-01-12)
# Week 1 = 2026-01-12 + 0-6 days, Week 2 = 2026-01-12 + 7-13 days, etc.
week_start_date = pd.Timestamp("2026-01-12")
mondays["days_since_week1_start"] = (mondays["article_date"] - week_start_date).dt.days
mondays["week"] = (mondays["days_since_week1_start"] // 7) + 1
# Explain missing weeks (MLK holiday on 1/19)
print("Note: MLK holiday on 1/19 causes week 2 to be missing")
# Filter to weeks >= 1 (exclude dates before 1/12, but no upper limit - open-ended)
mondays_filtered = mondays[mondays["week"] >= 1].copy()
print(f"Week start date: {week_start_date.date()}")
print(f"Week range in data: {mondays_filtered['week'].min()} to {mondays_filtered['week'].max()} (open-ended)")
print(f"\nMondays per week:")
week_counts = mondays_filtered.groupby("week")["article_date"].nunique()
print(week_counts)
print(f"\nMissing weeks (expected but not present):")
all_weeks = set(range(mondays_filtered['week'].min(), mondays_filtered['week'].max() + 1))
present_weeks = set(week_counts.index)
missing_weeks = sorted(all_weeks - present_weeks)
if missing_weeks:
    for w in missing_weeks:
        expected_date = week_start_date + pd.Timedelta(days=7*(w-1))
        print(f"  Week {w}: {expected_date.date()} (no articles on this Monday)")
        # Check if this date exists in the raw daily data (not just Mondays)
        if expected_date.date() in daily["article_date"].dt.date.values:
            print(f"    → Date exists in daily data but is not a Monday (dayofweek check)")
        else:
            print(f"    → Date not in daily data at all (no articles on this date)")
else:
    print("  None")

In [ ]:
# Summary statistics per ticker per week (Mondays only)
# Use mondays_filtered to exclude week 0 (dates before 1/12)
weekly_stats = mondays_filtered.groupby(["ticker", "week"]).agg(
    sentiment_mean=("sentiment_mean", "mean"),
    sentiment_median=("sentiment_median", "median"),
    # sentiment_std excluded: std of already-aggregated daily means is problematic (NaN when only one Monday)
    monday_count=("article_date", "nunique"),  # number of Mondays in this week with data
    total_articles=("article_count", "sum"),
).reset_index()

print("Summary statistics per ticker per week (Mondays only, weeks 1+):")
print("=" * 80)
weekly_stats

In [ ]:
# Pivot table for mean sentiment per ticker (per week) - weeks 1+ (open-ended)
weekly_pivot_mean = weekly_stats.pivot(index="ticker", columns="week", values="sentiment_mean")
print("Mean sentiment per ticker per week (Mondays only, weeks 1+) - pivot view:")
print("=" * 80)
print("Note: Week 0 (dates before 1/12) is excluded. If a ticker has NaN for a week,")
print("      that ticker had no articles on the Monday(s) in that week.")
print("      Weeks are open-ended; new weeks will appear as data extends past 2/9.")
weekly_pivot_mean


In [ ]:
# Pivot table for median sentiment per ticker (per week)
weekly_pivot_median = weekly_stats.pivot(index="ticker", columns="week", values="sentiment_median")
print("Median sentiment per ticker per week (Mondays only) - pivot view:")
print("=" * 70)
# Display table
weekly_pivot_median